# Insurance Charges Prediction Project

## Objective
To predict insurance charges using demographic and health-related data, leveraging supervised learning techniques.

## Problem Statement
How can we accurately estimate a customer’s insurance charges based on features like age, BMI, smoking status, and region?

## Hypotheses
1. Smoking status will be a major determinant of insurance charges.
2. Charges will increase with BMI and age.

## Success Metrics
- R² Score
- Mean Absolute Error (MAE)
- Root Mean Squared Error (MSE)

## Tools and Techniques
- **Tools**: Python, Jupyter Notebook, scikit-learn, matplotlib.
- **Techniques**: Regression models, feature engineering, and hyperparameter tuning.

## Workflow
1. Load and explore the data.
2. Perform EDA and visualization.
3. Preprocess the data.
4. Train multiple models and tune hyperparameters.
5. Evaluate the models using metrics.
6. Interpret the results and draw insights.
7. Summarize findings in a report.

LOADING THE DATASET

In [5]:
pip install openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 245.8 kB/s eta 0:00:01
   ------------ -------------------------- 81.9/250.9 kB 416.7 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 403.5 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 403.5 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 405.9 kB/s eta 0:00:01
   ---------------------------------- --- 225.3/250.9 kB 550.0 kB/s eta 0:00:01
   -------------------------------------  245.8/250.9 kB 537.7 kB/s eta 0:00:01
   -------------------------------------- 250.9/250.9 kB 497.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Load the dataset
file_path = "C:/Users/Admin/Desktop/Insurance_Charges_Prediction/insurance.xlsx"
insurance_df = pd.read_excel(file_path)